### Импорт библиотек

In [356]:
import pandas as pd
import numpy as np
import re

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from datetime import datetime

### Вывод дата фрейма

In [357]:
path = '/home/valerianikolaeva/Downloads/_data.csv'
df = pd.read_csv(path)
df.head()

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",...,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",...,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615


### Удаление строк с квартирами, находящимися не в Москве

In [358]:
df['Moscow']=df['Адрес'].apply(lambda x: x.split(',')[0])
df = df[df['Moscow']=='Москва']
df = df.reset_index(drop=True)

### Удаление колонок, не влияющих на цену квартиры

In [359]:
df.drop(['Тип', 'Название ЖК', 'Телефоны', 'Описание', 'Ссылка на объявление', 'Unnamed: 0', 'Серия дома', 'Moscow'], axis=1, inplace=True)

### 'Парковка'

In [360]:
df['Парковка'] = df['Парковка'].replace(to_replace ='открытая', value = 'бесплатная во дворе', regex = True)
df['Парковка'] = df['Парковка'].replace(to_replace ='наземная', value = 'платная во дворе', regex = True)
df['Парковка'] = df['Парковка'].fillna('бесплатная во дворе')

df['parking_encoded'] = df['Парковка'].map({val: idx for idx, val in enumerate(df['Парковка'].unique())})

### 'Количество комнат'

In [361]:
df["rooms"]=df["Количество комнат"].apply(lambda x: str(x).split(",")[0] if len(str(x).split(","))>1 else x )
df['rooms']=df['rooms'].apply(lambda x: int(x) if str(x).isdigit() else x)
df.fillna({'rooms':df['rooms'].median()}, inplace=True) #заполняем пустые данные по комнатам
df['rooms'] = df['rooms'].astype('int')

### 'Дом'

In [362]:
df["building_type"]=df["Дом"].apply(lambda x: x.split(",")[1].strip() if len(x.split(","))>1 else None)
df["floor"]=df["Дом"].apply(lambda x: int(x.split(",")[0].split("/")[0]) if len(x.split(","))>1 else int(x.split("/")[0]))
df["totalfloors"]=df["Дом"].apply(lambda x: int(x.split(",")[0].split("/")[1]) if len(x.split(","))>1 else int(x.split("/")[1]))
str(df['building_type'].mode())[0]
df['building_type'].mode()
df.fillna({'building_type':"Панельный"}, inplace=True) # заполнил модой, в идеале можно было заполнить по распределению по существующим домам
dict1={'Монолитный':1, 'Монолитно-кирпичный':2, 'Панельный':3, 'Кирпичный':5, 'Сталинский':4, 'старый фонд':6, 'Блочный':7, 'Деревянный':8, 'Щитовой':9}
df["building_type"]=df["building_type"].map(dict1)

df["totalfloors"].mean()
df["floor"].mean()

7.520747834017328

### 'Лифт'

In [363]:
df["Лифт"].unique()
df['elevator']=df['Лифт'].apply(lambda x: sum([int(i) for i in str(x) if i.isdigit()]))
df["elevator"].unique()
df["elevator"].mean()

1.5788620357703804

### 'Ремонт'

In [364]:
df.fillna({'Ремонт':"Косметический"}, inplace=True)
dict={"Без ремонта":0, "Косметический":1,"Евроремонт":2,"Дизайнерский":3}
df["design"]=df["Ремонт"].map(dict)

### 'Санузел' (Совмещенный/Раздельный) + количество санузлов

In [365]:
df.fillna({'Санузел':"Совмещенный (1)"}, inplace=True)
df["sep_bathroom"]=df["Санузел"].apply(lambda x: 1 if "Раздельный" in x else 0)
df['bathrooms']=df['Санузел'].apply(lambda x: sum([int(i) for i in str(x) if i.isdigit()]))

### 'Метро'

In [366]:
df['Минуты до метро'] = df['Метро'].str.extract('(\d+)').astype(float)

In [367]:
def distance_multiplier(row): 
    if pd.notna(row['Метро']):
        if 'пешком' in row['Метро']: 
            return row['Минуты до метро'] * 120 
        elif 'на машине' in row['Метро']: 
            return row['Минуты до метро'] * 650 
    return None

df['Distance_to_Metro'] = df.apply(distance_multiplier, axis=1)


In [368]:
df.drop(['Минуты до метро'], axis=1, inplace=True)

In [369]:
df['Distance_to_Metro'] = df['Distance_to_Metro']. fillna(df['Distance_to_Metro'].mean())

### Колонка 'Площадь комнат'

In [370]:
df.drop(['Площадь комнат, м2'], axis=1, inplace=True)

### Колонка 'Можно с детьми/животными'

In [371]:
df['kids_friendly'] = df['Можно с детьми/животными'].apply(lambda x: 1 if 'Можно с детьми' in str(x) else 0) 
df['animal_friendly'] = df['Можно с детьми/животными'].apply(lambda x: 1 if 'Можно с животными' in str(x) else 0)

### 'Мусоропровод', 'Окна', 'Площадь, м2'

In [372]:
df["square"] = [x.split('/')[0] for x in df['Площадь, м2']] #

okna_to_num = {"Во двор": 0,"На улицу": 1,"На улицу и двор": 2}
musor_to_bool = {"Да":1,"Нет":0}

df["okna_coded"] = df["Окна"].map(okna_to_num) #
df["musor_coded"] = df["Мусоропровод"].map(musor_to_bool) #

df["clear_price"] = [x.split("/")[0] for x in df['Цена']] #

df["currency"] = [x.split(" ")[1] for x in df["clear_price"]] #
df["clear_price"] = [x.split(" ")[0] for x in df["clear_price"]] #

df["clear_price"] = df["clear_price"].astype(float) #делаю числа
df["square"] = df["square"].astype(float) #делаю числа

df["musor_coded"] = df["musor_coded"].fillna(0)
df["square"] = df["square"].fillna(round(df["square"].mean()))
df["okna_coded"] = df["okna_coded"].fillna(round(df["okna_coded"].mean()))

### 'Цена'

In [373]:
def extract_price_info(row):
    #цена за месяц и залог
    price_monthly, deposit = re.findall(r'(\d+\.\d+)\sруб\./\s*За\sмесяц,\s*Залог\s*-\s*(\d+)\sруб', row['Цена'])[0] if re.findall(r'(\d+\.\d+)\sруб\./\s*За\sмесяц,\s*Залог\s*-\s*(\d+)\sруб', row['Цена']) else (0, 0)
    #коммуналка
    utilities = 1 if 'Коммунальные услуги включены' in row['Цена'] else 0
    #количество месяцев предоплаты
    prepayment_months = re.findall(r'Предоплата\s(\d+)\sмес', row['Цена'])
    prepayment_months = int(prepayment_months[0]) if prepayment_months else 0
    return price_monthly, deposit, utilities, prepayment_months

df[['Цена за месяц','Залог','Коммунальные услуги включены', 'Предоплата мес']] = df.apply(extract_price_info, axis=1,result_type='expand')
df.drop('Цена', axis=1, inplace=True)

### 'Дополнительно'

In [374]:
def is_furnished(row):
    details = str(row['Дополнительно'])  # Convert to string to handle NaN values
    if 'Мебель в комнатах' in details and 'Мебель на кухне' in details and 'Ванна' in details:
        return 'Заставлена'
    else:
        return 'Не заставлена'

# Apply the function to each row of the dataframe and create a new column
df['Заставлена мебелью'] = df.apply(is_furnished, axis=1)
df['furnished'] = df['Заставлена мебелью'].apply(lambda x: 1 if x == 'Заставлена' else 0)
df.drop(['Заставлена мебелью'], axis=1, inplace=True)

In [375]:
df

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Ремонт,Балкон,Окна,...,square,okna_coded,musor_coded,clear_price,currency,Цена за месяц,Залог,Коммунальные услуги включены,Предоплата мес,furnished
0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,Дизайнерский,NaN,NaN,...,200.0,1.0,1.0,500000.0,руб.,500000.0,500000,1,1,1
1,271634126,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,Дизайнерский,NaN,На улицу и двор,...,198.0,2.0,0.0,500000.0,руб.,500000.0,500000,1,1,1
2,271173086,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,Евроремонт,NaN,На улицу и двор,...,200.0,2.0,0.0,500000.0,руб.,500000.0,500000,0,1,1
3,272197456,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,Евроремонт,NaN,На улицу и двор,...,170.0,2.0,0.0,400000.0,руб.,400000.0,400000,0,1,1
4,273614615,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",бесплатная во дворе,Евроремонт,NaN,На улицу и двор,...,58.0,2.0,1.0,225000.0,руб.,225000.0,225000,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19732,215565511,NaN,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,бесплатная во дворе,Евроремонт,Балкон (1),NaN,...,35.0,1.0,0.0,42000.0,руб.,42000.0,42000,1,1,1
19733,274654844,1,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",бесплатная во дворе,Евроремонт,Лоджия (1),Во двор,...,38.7,0.0,0.0,45000.0,руб.,45000.0,45000,1,1,1
19734,268679909,"2, Оба варианта",м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",бесплатная во дворе,Дизайнерский,Балкон (1),На улицу и двор,...,43.1,2.0,0.0,50000.0,руб.,50000.0,50000,1,1,0
19735,274807525,2,м. Солнцево (11 мин пешком),"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",52.5/10.0,"8/23, Монолитный",платная во дворе,Евроремонт,Лоджия (1),Во двор,...,52.5,0.0,1.0,55000.0,руб.,55000.0,50000,1,2,1


### 'Балкон'

In [376]:
df['Балкон'] = df["Балкон"].fillna('Нет инфо')
df['Balcony'] = df['Балкон'].str.contains('Балкон|Лоджия').map({True: 1, False: 0})

In [377]:
translations = {
    'Цена за месяц': 'Price per month',
    'Залог': 'Deposit',
    'Коммунальные услуги включены': 'Utilities included',
    'Предоплата мес': 'Prepayment months',
    'Заставлена мебелью': 'Furnished'
}

df.rename(columns=translations, inplace=True)

In [378]:
df = df.loc[df['currency'] == '$']
df = df.dropna(subset=['Метро'])

In [379]:
df

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Ремонт,Балкон,Окна,...,okna_coded,musor_coded,clear_price,currency,Price per month,Deposit,Utilities included,Prepayment months,furnished,Balcony
356,255388888,4,м. Проспект Мира (5 мин пешком),"Москва, Протопоповский переулок, 17С3",160.0,"5/14, Монолитный",бесплатная во дворе,Косметический,Лоджия (1),Во двор,...,0.0,1.0,5000.0,$,0,0,1,1,1,1
443,254126155,4,м. Проспект Мира (4 мин пешком),"Москва, Протопоповский переулок, 17С3",162.0/15.0,"4/14, Монолитно-кирпичный",подземная,Евроремонт,Балкон (1),Во двор,...,0.0,0.0,5000.0,$,0,0,0,1,0,1
444,255696192,4,м. Проспект Мира (4 мин пешком),"Москва, Протопоповский переулок, 17С3",162.0,"5/14, Монолитный",подземная,Дизайнерский,Балкон (1),NaN,...,1.0,1.0,5000.0,$,0,0,1,1,1,1
10101,237916979,3,м. Проспект Вернадского (20 мин пешком),"Москва, улица Новаторов, 8К2",163.0/79.0/15.0,"10/20, Монолитный",подземная,Дизайнерский,Лоджия (1),На улицу и двор,...,2.0,1.0,5000.0,$,0,0,1,1,1,1
11539,234691832,6,м. Новые Черемушки (9 мин на машине),"Москва, улица Академика Пилюгина, 24К1",400.0/320.0,23/23,бесплатная во дворе,Евроремонт,"Балкон (1), Лоджия (1)",На улицу и двор,...,2.0,0.0,10000.0,$,0,0,1,1,1,1
11540,239486235,6,м. Калужская (None мин пешком),"Москва, улица Академика Пилюгина, 24К1",400.0/46.0,23/23,подземная,Дизайнерский,Балкон (1),На улицу и двор,...,2.0,0.0,10000.0,$,0,0,0,0,1,1
12722,239268059,5,м. Парк Культуры (7 мин пешком),"Москва, Еропкинский переулок, 16/23",270.0,"5/7, Монолитно-кирпичный",бесплатная во дворе,Дизайнерский,Нет инфо,NaN,...,1.0,1.0,25000.0,$,0,0,0,1,1,0
12794,273034278,5,м. Кропоткинская (None мин пешком),"Москва, Бутиковский переулок, 3",290.0/215.0/30.0,6/6,бесплатная во дворе,Дизайнерский,"Балкон (1), Лоджия (1)",NaN,...,1.0,0.0,25000.0,$,0,0,0,1,0,1
13850,224335547,3,м. Проспект Вернадского (6 мин на машине),"Москва, улица Новаторов, 8К2",160.0/128.0,"10/14, Монолитный",бесплатная во дворе,Евроремонт,Лоджия (1),На улицу и двор,...,2.0,0.0,5000.0,$,0,0,1,1,1,1
17345,270187964,"5, Оба варианта",м. Маяковская (7 мин пешком),"Москва, Краснопролетарская улица, 7",250.0/201.0/18.0,4/19,подземная,Евроремонт,Нет инфо,На улицу и двор,...,2.0,0.0,10638.0,$,0,0,0,1,1,0


In [380]:
df.to_csv('data.csv', index=False)